이름: 이예주

학번: 20193237

# Neural networks with PyTorch

Pytorch의 `nn.module`을 활용하여 만드는 유용한 방법을 학습합니다.

<div style="text-align:center"><img src='https://drive.google.com/uc?export=download&id=1J2SeiPpVJs1-ML2BdLrcxkGGmHpRxIVE' width="250" height="200"> 

### Lego block coding! </div>

In [1]:
import torch
import numpy as np

from torch import nn  # ReLU, 시그모이드 함수들 존재~~
import torch.nn.functional as F # ""
from collections import OrderedDict # 순서가 있는 딕셔너리(파이썬 라이브러리)

torch.manual_seed(0)

layer별로 Linear연산을 할 수 있어야 함.

`nn.Linear`: $Z^{[\ell]} = A^{[\ell-1]}W^T+b$
연산.

해당 layer의 

- 입력 차원 `n_input=30`  # 유닛 수(동그라미 수)
- 출력 차원 `n_output=60`


--- 메모 ---   
```python
Z[ℓ] = A[ℓ-1] * W.T + b
      = (batch_size, n_input) * (n_output, n_input).T + (n_output,)
      = (batch_size, n_output)

```
**`batch_size`는 입력 데이터의 샘플 수**   
**W의 크기는 (n_input, n_output)**   
<br><br>
해당 layer의 동그라미 수가 30개, 나가는 수가 60개   
* Z의 차원   
column의 개수가 60(출력겸 다음 layer의 입력)   
row들의 개수는 batch(입력 데이터의 샘플)수   
* A의 차원   
열의 개수가 30(입력)   
행의 수는 batch 수   
* W의 차원   
행60, 열30을 바꿔줬기 때문에(transpose)   
행 : 30   
열 : 60   
* b의 차원   
output 출력과 개수가 같아야 한다.

In [2]:
# Example of nn.linear
# Linear 연산하는 함수
linear_layer1 = nn.Linear(30, 60)  # nn.Linear(입력수, 출력수)  / 들어가는 동그라미 수, 나가는 동그라미 수

# batch는(batch 수의 차원은) 언제나 동일하다.(학습 중에 일정하게 유지됨) 

# 입력받는 것의 shape 0번이 batch size라는 것을 알고 있기 때문에 batch size는 따로 입력 X.

`nn.Linear`는 W와 b값을 랜덤으로 초기화 해서 가지고 있다.

In [3]:
# test sample
A = torch.randn(60, 30)  # 배치사이즈는 60
linear_layer1(A).shape  # (60, 30) x (30, 60) = (60, 60)

torch.Size([60, 60])

How to get the weights and bias of each `nn.Linear`

In [4]:
linear_layer1.weight 

Parameter containing:
tensor([[-0.0014,  0.0979, -0.1503,  ..., -0.1694, -0.1149, -0.0462],
        [-0.0712,  0.1577, -0.1183,  ..., -0.1416, -0.1265, -0.0943],
        [ 0.0826,  0.0734, -0.1081,  ..., -0.0836,  0.0702, -0.1081],
        ...,
        [ 0.1375,  0.0413, -0.0681,  ..., -0.0763,  0.1550,  0.0914],
        [-0.1336,  0.1579, -0.0027,  ...,  0.1744,  0.0625, -0.0929],
        [-0.1096, -0.0640, -0.0572,  ...,  0.0878,  0.0632, -0.0053]],
       requires_grad=True)

In [5]:
linear_layer1.weight.shape  # W의 transpose를 가지고 있다.

torch.Size([60, 30])

In [6]:
linear_layer1.bias.shape  # bias는 1차원 / output layer가 60이니까..!

torch.Size([60])

In [7]:
# Example of weights
# weight를 다른 값으로 바꾸고 싶을 때 사용.
linear_layer1.weight.data = torch.ones_like(linear_layer1.weight)  # torch.ones_like :입력 텐서와 동일한 모양(차원)의 텐서를 만들어주는데, 1로 채워줘

In [8]:
linear_layer1.weight.data

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

### NN example

- input units: 20
- hidden layer: 30, 40
- output units: 3
- activation function: ReLU
- output layer: No activation (마지막 layer만)



In [9]:
# Simple NN construction

class FCN(nn.Module):      # nn.Module : neural network모듈을 만들기 위한 베이스 모듈(class).
    def __init__(self):    # 자동으로 실행되는 첫번째 함수
        super().__init__() # nn.Module을 인스턴스로 불러온 것. 그리고 그것의 __init__함수를 실행시켜라.
        
        self.lin1 = nn.Linear(20, 30)
        self.lin2 = nn.Linear(30, 40)
        self.lin3 = nn.Linear(40, 3)
        
        self.relu = nn.ReLU(True)   # F.relu(True)
        
    # 위의 것들을 가지고 forward pass연산을 하는 것을 정의하는 함수(이름은 forward로 고정해야함)
    # self를 해주는 이유 -> 위에 있는 함수를 불러와줘야하기 때문에.(다 일맥상통 함)
    def forward(self, x):  # 입력 : x
        x = self.lin1(x)   # x를 집어넣어서 Linear 연산을 한 다음에 x로 받는다
        x = self.relu(x)   # relu 씌우기
        x = self.lin2(x)
        x = self.relu(x)
        x = self.lin3(x)   # 마지막 layer는 No activation.
        
        return x


In [10]:
Xtrain = torch.randn(60, 20)  # 입력 샘플(배치수:60)

model = FCN()     # 인스턴스를 만들고
# model(Xtrain)   # 데이터를 집어넣으면 forward연산이 된다 

# model.lin1.weight.data = torch.ones(30,20)  # ones에는 weight의 차원에 맞춰서 입력

In [11]:
model

FCN(
  (lin1): Linear(in_features=20, out_features=30, bias=True)
  (lin2): Linear(in_features=30, out_features=40, bias=True)
  (lin3): Linear(in_features=40, out_features=3, bias=True)
  (relu): ReLU(inplace=True)
)

In [12]:
# lin1에 들어있는 weight들
model.lin1.weight 

Parameter containing:
tensor([[ 1.1542e-01, -1.5343e-01, -1.4473e-01,  6.0137e-02,  3.4798e-02,
         -1.9658e-01, -1.6290e-01, -1.7801e-01,  1.7759e-01,  1.4619e-01,
         -1.1770e-01,  3.3738e-02, -5.1431e-02,  1.9439e-01, -2.0253e-01,
         -1.6313e-01, -1.8617e-01,  1.3013e-01, -1.1178e-01,  1.3318e-01],
        [ 7.5586e-02, -1.0930e-01, -6.0237e-02, -9.0490e-02,  9.8650e-02,
         -8.9674e-02,  7.5885e-02,  1.5974e-02, -1.7857e-01, -1.4056e-01,
         -1.5504e-01,  1.1717e-01,  4.8451e-02, -5.6224e-03, -1.3314e-01,
         -2.2155e-01, -7.9131e-02, -1.7114e-01, -2.2241e-01, -2.1327e-01],
        [-1.4606e-01, -2.1738e-01,  6.9704e-02,  8.3668e-02,  9.0129e-02,
         -2.2260e-01,  1.3827e-01,  4.3595e-02,  1.1434e-01, -5.4957e-02,
          1.0738e-01, -2.1207e-01,  7.6162e-02,  1.7731e-01,  9.3997e-02,
         -2.1077e-01, -6.2772e-02, -2.7768e-02,  1.1716e-01, -8.4613e-02],
        [ 3.2161e-02, -7.8497e-02, -3.5522e-02, -4.4512e-03, -1.2840e-01,
         -1.5

In [13]:
# 그 텐서를 보는 것 (30x20)
model.lin1.weight.data 

tensor([[ 1.1542e-01, -1.5343e-01, -1.4473e-01,  6.0137e-02,  3.4798e-02,
         -1.9658e-01, -1.6290e-01, -1.7801e-01,  1.7759e-01,  1.4619e-01,
         -1.1770e-01,  3.3738e-02, -5.1431e-02,  1.9439e-01, -2.0253e-01,
         -1.6313e-01, -1.8617e-01,  1.3013e-01, -1.1178e-01,  1.3318e-01],
        [ 7.5586e-02, -1.0930e-01, -6.0237e-02, -9.0490e-02,  9.8650e-02,
         -8.9674e-02,  7.5885e-02,  1.5974e-02, -1.7857e-01, -1.4056e-01,
         -1.5504e-01,  1.1717e-01,  4.8451e-02, -5.6224e-03, -1.3314e-01,
         -2.2155e-01, -7.9131e-02, -1.7114e-01, -2.2241e-01, -2.1327e-01],
        [-1.4606e-01, -2.1738e-01,  6.9704e-02,  8.3668e-02,  9.0129e-02,
         -2.2260e-01,  1.3827e-01,  4.3595e-02,  1.1434e-01, -5.4957e-02,
          1.0738e-01, -2.1207e-01,  7.6162e-02,  1.7731e-01,  9.3997e-02,
         -2.1077e-01, -6.2772e-02, -2.7768e-02,  1.1716e-01, -8.4613e-02],
        [ 3.2161e-02, -7.8497e-02, -3.5522e-02, -4.4512e-03, -1.2840e-01,
         -1.5520e-01,  1.8460e-01, 

In [14]:
# bias
model.lin1.bias

Parameter containing:
tensor([ 0.1095,  0.0836,  0.1833, -0.2151, -0.1392, -0.0284, -0.0425, -0.1485,
        -0.2144, -0.1708, -0.0347,  0.1898,  0.1482,  0.0645, -0.1373, -0.0900,
         0.0215, -0.2073, -0.0102,  0.0455,  0.2022, -0.0587,  0.2195,  0.0169,
        -0.1551,  0.2113,  0.1939, -0.1767, -0.1297,  0.1069],
       requires_grad=True)

In [15]:
# Example of parameters() in models
# iterator(모든 파라미터) 출력
param_iterator = model.parameters()   

# 위의 network에 있는 모든 파라미터를 for문을 통해서 출력함.
for param in param_iterator:
    print(param)

    

# 첫번째 layer의 W
# 첫번째 layer의 bias
# 두번째 layer의 W
# 두번째 layer의 bias
# ...

Parameter containing:
tensor([[ 1.1542e-01, -1.5343e-01, -1.4473e-01,  6.0137e-02,  3.4798e-02,
         -1.9658e-01, -1.6290e-01, -1.7801e-01,  1.7759e-01,  1.4619e-01,
         -1.1770e-01,  3.3738e-02, -5.1431e-02,  1.9439e-01, -2.0253e-01,
         -1.6313e-01, -1.8617e-01,  1.3013e-01, -1.1178e-01,  1.3318e-01],
        [ 7.5586e-02, -1.0930e-01, -6.0237e-02, -9.0490e-02,  9.8650e-02,
         -8.9674e-02,  7.5885e-02,  1.5974e-02, -1.7857e-01, -1.4056e-01,
         -1.5504e-01,  1.1717e-01,  4.8451e-02, -5.6224e-03, -1.3314e-01,
         -2.2155e-01, -7.9131e-02, -1.7114e-01, -2.2241e-01, -2.1327e-01],
        [-1.4606e-01, -2.1738e-01,  6.9704e-02,  8.3668e-02,  9.0129e-02,
         -2.2260e-01,  1.3827e-01,  4.3595e-02,  1.1434e-01, -5.4957e-02,
          1.0738e-01, -2.1207e-01,  7.6162e-02,  1.7731e-01,  9.3997e-02,
         -2.1077e-01, -6.2772e-02, -2.7768e-02,  1.1716e-01, -8.4613e-02],
        [ 3.2161e-02, -7.8497e-02, -3.5522e-02, -4.4512e-03, -1.2840e-01,
         -1.5

## nn.Sequential() example

In [16]:
# nn.Sequential() example

class FCN_seq(nn.Module):
    def __init__(self):     # 초기화 함수 정의
        super().__init__()  # super의 __init__ 실행
        
        # 순차적으로 내가 실행할 모듈들을 정의한다.
        # Sequential 함수는 forward함수가 내부적으로 정의되어 있다.
        self.fc = nn.Sequential(nn.Linear(20, 30),
                      nn.ReLU(True),
                      nn.Linear(30, 40),
                      nn.ReLU(True),
                      nn.Linear(40, 3)   # 마지막 layer는 Linear layer로
        )
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [17]:
model_seq = FCN_seq()  # 인스턴스 생성
print(Xtrain.shape)
print(model_seq)
print(model_seq(Xtrain))
# print(model_seq.fc(Xtrain))  -> forward함수 정의 안 했을 때, fc를 붙인다.

torch.Size([60, 20])
FCN_seq(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
)
tensor([[ 0.1613,  0.0474,  0.1423],
        [ 0.1216,  0.0780,  0.0733],
        [ 0.1969,  0.1477,  0.0351],
        [ 0.1259,  0.0780,  0.1492],
        [ 0.0796,  0.1557, -0.0115],
        [ 0.1025,  0.0723,  0.1972],
        [ 0.1049,  0.2304, -0.0196],
        [ 0.2009,  0.1772,  0.0686],
        [ 0.0065,  0.3023,  0.1493],
        [ 0.2442, -0.0287,  0.2737],
        [ 0.2066,  0.1528,  0.0567],
        [ 0.2014,  0.2286,  0.1846],
        [ 0.2565,  0.0261,  0.0793],
        [ 0.2581,  0.1663,  0.1000],
        [ 0.2335,  0.3101,  0.0227],
        [ 0.1506,  0.0258,  0.1484],
        [ 0.1547,  0.1805, -0.0363],
        [ 0.1707,  0.2219, -0.0271],
        [ 0.2267,  0.1917,  0.0362],
      

In [18]:
Xtrain.shape

torch.Size([60, 20])

In [19]:
model_seq.fc[0]

Linear(in_features=20, out_features=30, bias=True)

### 다른 NN 방법(class 사용x)

In [20]:
# 또는
model_by_sequential = nn.Sequential(nn.Linear(20, 30),
                      nn.ReLU(True),
                      nn.Linear(30, 40),
                      nn.ReLU(True),
                      nn.Linear(40, 3)   # 마지막 layer는 Linear layer로
        )

In [21]:
model_by_sequential(Xtrain)

tensor([[-0.1290,  0.1012, -0.0432],
        [-0.1807,  0.0422, -0.0590],
        [-0.0867,  0.0434, -0.0832],
        [-0.1133,  0.1099, -0.0820],
        [-0.1334,  0.0641, -0.1664],
        [-0.2149,  0.0981, -0.1222],
        [-0.0897,  0.0634, -0.0132],
        [-0.0316,  0.1087, -0.0741],
        [-0.1536,  0.1385, -0.1129],
        [-0.0053,  0.0458, -0.1444],
        [-0.1036,  0.0451, -0.0375],
        [-0.0530,  0.2289, -0.0727],
        [-0.0994,  0.1190, -0.0743],
        [-0.0525,  0.1025, -0.0331],
        [-0.0651, -0.0479, -0.1240],
        [-0.0760,  0.0858, -0.0104],
        [-0.0589,  0.1708, -0.1553],
        [-0.1079,  0.0682,  0.0143],
        [-0.0809,  0.0537, -0.0853],
        [-0.1456,  0.0114, -0.0614],
        [-0.0732,  0.0301, -0.0336],
        [-0.1034,  0.0222, -0.1068],
        [-0.0352,  0.0648, -0.0863],
        [-0.0185,  0.0939, -0.1200],
        [-0.1203,  0.0337,  0.0012],
        [-0.0015,  0.1213, -0.0419],
        [-0.0330,  0.1190, -0.0865],
 

In [22]:
class FCN_seq_v2(nn.Module):
    def __init__(self):
        super().__init__()
        

        # self.fcn_block(20, 30)가 두 개의 list를 리턴 + 리스트에 리스트를 더해서 아이템들을 추가
        temp = self.fcn_block(20, 30)+self.fcn_block(30, 40)+[nn.Linear(40,1)]   # 마지막은 linear layer로 예외처리 해줌
        self.fc = nn.Sequential(*temp)  # 리스트를 풀어서 안에 있는 것을 나열한다.
        
        
    def fcn_block(self, in_dim, out_dim):
        return [nn.Linear(in_dim, out_dim),  # nn.Sequential을 하지 않고 두개의 list를 return한다.
                             nn.ReLU(True)]
        
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [23]:
model_seq_v2 = FCN_seq_v2()  # 인스턴스 생성
print(model_seq_v2)
print(model_seq_v2(Xtrain))  # forward함수가 잘 동작하는지

FCN_seq_v2(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=1, bias=True)
  )
)
tensor([[ 0.0024],
        [-0.0164],
        [ 0.0110],
        [ 0.0904],
        [ 0.0215],
        [ 0.0451],
        [ 0.1046],
        [-0.0817],
        [-0.0119],
        [-0.0311],
        [ 0.0728],
        [-0.0408],
        [ 0.0488],
        [ 0.0549],
        [ 0.0904],
        [ 0.0783],
        [-0.0463],
        [ 0.0951],
        [-0.0649],
        [-0.0142],
        [ 0.0607],
        [ 0.0439],
        [-0.0259],
        [-0.0705],
        [ 0.0784],
        [ 0.1228],
        [-0.0202],
        [ 0.1298],
        [ 0.0849],
        [ 0.0272],
        [-0.0376],
        [ 0.0315],
        [ 0.0046],
        [ 0.0089],
        [ 0.0573],
        [ 0.0502],
        [ 0.0856],
        [ 0.0536],
     

### zip함수
각 리스트마다 순차적으로 쌍으로 묶어준다. (같은 인덱스끼리)

In [28]:
hlayer1 = [30, 40, 50, 60]
hlayer2 = [20, 80, 99, 100]

for i, j in zip(hlayer1, hlayer2):
    print('i=', i, ' j=', j)

i= 30  j= 20
i= 40  j= 80
i= 50  j= 99
i= 60  j= 100


In [5]:
# hlayer0 = [30, 40, 50, 60, 80, 90]

# for i, j in zip(hlayer0[:3], hlayer0[3:]):
#     print('i=', i, ' j=', j)

i= 30  j= 60
i= 40  j= 80
i= 50  j= 90


In [29]:
class FCN_final(nn.Module):
    def __init__(self, in_dim, hlayer, out_dim):   # hidden layer의 수를 list로 받는다 -> 일반적인 list를 준 것에 대해서 network를 만든다
        super().__init__()
        
        l_list = self.fcn_block(in_dim, hlayer[0])  # 첫번째 layer는 따로 만들어줌
        
        for l1, l2 in zip(hlayer[:-1], hlayer[1:]):  # zip(마지막것만 빼라, 첫번째것만 빼라)
            l_list = l_list + self.fcn_block(l1, l2)
        
        l_list = l_list + [nn.Linear(hlayer[-1], out_dim)]  # 마지막 layer는 예외처리로 직접 작성
                                                            # Linear는 list가 아니니까 []를 씌워주고 입력
        
        self.fc = nn.Sequential(*l_list)
        
        
    def fcn_block(self, in_dim, out_dim):
        return [nn.Linear(in_dim, out_dim),
                             nn.ReLU(True)]
        
        
    def forward(self, x):
        return self.fc(x)
        
        

In [30]:
hlayer = [30, 40]
in_dim = 20
out_dim= 3

myfcn_final = FCN_final(in_dim, hlayer, out_dim)

In [31]:
myfcn_final

FCN_final(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
)

## Ordered dict example

**<이름으로 부르면 좋은점>**   
1. 사람들한테 설명하는 데 편리하다.
2. 어떤 이름에 있는 것들만 교체하는거 가능하다.

In [32]:
# Ordered dict example
# nn.Sequential() example

class FCN_seq_ordered_dic(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc = nn.Sequential(OrderedDict([('lin1', nn.Linear(20, 30)),  # key와 value 만들기 위해 튜플로 작성
                      ('relu1', nn.ReLU(True)),
                      ('lin2', nn.Linear(30, 40)),
                      ('relu2',nn.ReLU(True)),
                      ('lin3', nn.Linear(40, 3))
                                            ])
        )
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [33]:
# ModuleList(), ModuleDict()

In [35]:
model_FCN_seq_ordered_dic = FCN_seq_ordered_dic()
model_FCN_seq_ordered_dic

FCN_seq_ordered_dic(
  (fc): Sequential(
    (lin1): Linear(in_features=20, out_features=30, bias=True)
    (relu1): ReLU(inplace=True)
    (lin2): Linear(in_features=30, out_features=40, bias=True)
    (relu2): ReLU(inplace=True)
    (lin3): Linear(in_features=40, out_features=3, bias=True)
  )
)

In [37]:
model_FCN_seq_ordered_dic.fc.lin1  # lin1처럼 이름으로 부를 수도 있다.

Linear(in_features=20, out_features=30, bias=True)

모델 학습을 다 끝내놓고 network를 저장할 때 `model.state_dict()`를 저장한다.

### state_dict() 
모델을 다 설명해주는 것.(layer별 이름, 모듈들의 이름, 파라미터, bias...)

In [34]:
# state_dict() example
# 모델을 다 설명해주는 것.(layer별 이름, 모듈들의 이름, 파라미터, bias...)
model.state_dict()

OrderedDict([('lin1.weight',
              tensor([[ 1.1542e-01, -1.5343e-01, -1.4473e-01,  6.0137e-02,  3.4798e-02,
                       -1.9658e-01, -1.6290e-01, -1.7801e-01,  1.7759e-01,  1.4619e-01,
                       -1.1770e-01,  3.3738e-02, -5.1431e-02,  1.9439e-01, -2.0253e-01,
                       -1.6313e-01, -1.8617e-01,  1.3013e-01, -1.1178e-01,  1.3318e-01],
                      [ 7.5586e-02, -1.0930e-01, -6.0237e-02, -9.0490e-02,  9.8650e-02,
                       -8.9674e-02,  7.5885e-02,  1.5974e-02, -1.7857e-01, -1.4056e-01,
                       -1.5504e-01,  1.1717e-01,  4.8451e-02, -5.6224e-03, -1.3314e-01,
                       -2.2155e-01, -7.9131e-02, -1.7114e-01, -2.2241e-01, -2.1327e-01],
                      [-1.4606e-01, -2.1738e-01,  6.9704e-02,  8.3668e-02,  9.0129e-02,
                       -2.2260e-01,  1.3827e-01,  4.3595e-02,  1.1434e-01, -5.4957e-02,
                        1.0738e-01, -2.1207e-01,  7.6162e-02,  1.7731e-01,  9.3997e-02,
 

# Problem Setup

<div style="text-align:center"> <img src='https://drive.google.com/uc?export=download&id=1FRhniwGeeutBSJQRdW6GzshMfDrPz7oJ' width="250" height="200"> </div>
    
Build a Fully connected neural network with

- 3 layers
- 마지막 layer의 unit 수는 `1` 
  - 마지막 layer의 activation은 없음 (linear layer)
- Data feature 수는 `100` (입력 data의 feature수)

- input unit 수는 data 크기를 보고 맞추세요
- hidden layer의 unit 수는 `[80, 50]`
  - hidden layer의 activation 함수는 ReLU

- model class 명 `myFCN`
  - instance 명 `my_model` 생성
  - `my_model` 출력

## Problem 1

problem setup에서 구성한 neural network을 `nn.Sequential`을 활용하여 생성하세요

In [38]:
## 사용할 data 
batch_size = 30
num_feature = 100

X_train = torch.randn(batch_size, num_feature)

In [39]:
X_train.shape

torch.Size([30, 100])

In [46]:
# Problem 1 코딩 (매 줄마다 주석 필수 )

class myFCN(nn.Module):    # nn.Module : neural network모듈을 만들기 위한 베이스 모듈.
    def __init__(self):    # 초기화 함수 정의 
        super().__init__() # nn.Module을 인스턴스로 불러온 것 = super() / super의 __init__함수를 실행시키는 코드
        
        # 순차적으로 내가 실행할 모듈들을 정의한다.
        # Sequential 함수는 forword함수가 내부적으로 정의되어 있음
        self.fc = nn.Sequential(nn.Linear(100, 80),  # nn.Linear(num_feature, 80)
                                nn.ReLU(True),
                                nn.Linear(80, 50),   # hidden layer
                                nn.ReLU(True),
                                nn.Linear(50, 1)     # 마지막 layer는 Linear layer로
        )
        
        # 위의 것들을 가지고 forward pass연산을 하는 것을 정의하는 함수
        def forward(self, x):
            return self.fc(x)

In [47]:
# instance 생성
my_model = myFCN()

In [48]:
my_model

myFCN(
  (fc): Sequential(
    (0): Linear(in_features=100, out_features=80, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=80, out_features=50, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=50, out_features=1, bias=True)
  )
)

## Problem 2

problem setup에서 구성한 neural network을 `OrderedDict`을 활용하여 생성하세요
- 각 layer의 이름을 주고 생성하세요

In [55]:
# 답작성

class myFCN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # OrderedDict() : 순서를 기억하는 딕셔너리 자료형
        self.fc = nn.Sequential(OrderedDict([('lin1', nn.Linear(100, 80)),  # key와 value 만들기 위해 튜플로 작성
                      ('relu1', nn.ReLU(True)),
                      ('lin2', nn.Linear(80, 50)),
                      ('relu2',nn.ReLU(True)),
                      ('lin3', nn.Linear(50, 1))
                                            ])
        )
        
    def forward(self, x):
        return self.fc(x)

# Problem1에서 달라진 점은 각 layer에 이름을 준 것.

In [56]:
# instance 생성
my_model = myFCN()

In [57]:
my_model

myFCN(
  (fc): Sequential(
    (lin1): Linear(in_features=100, out_features=80, bias=True)
    (relu1): ReLU(inplace=True)
    (lin2): Linear(in_features=80, out_features=50, bias=True)
    (relu2): ReLU(inplace=True)
    (lin3): Linear(in_features=50, out_features=1, bias=True)
  )
)